<a href="https://colab.research.google.com/github/HNR1/Bachelors-project/blob/main/diffusers_test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/HNR1/pytorch-fid.git
!pip install transformers diffusers tomesd accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/HNR1/pytorch-fid.git to /tmp/pip-req-build-hpx93ich
  Running command git clone --filter=blob:none --quiet https://github.com/HNR1/pytorch-fid.git /tmp/pip-req-build-hpx93ich
  Resolved https://github.com/HNR1/pytorch-fid.git to commit bd3d7be6a734e946c8686e50972250ec87416937
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from diffusers import DiffusionPipeline
import torch, tomesd
import pytorch_fid.fid_score as fid
from pytorch_fid.inception import InceptionV3
import os.path

In [7]:
if os.path.exists("/content/Bachelors-project"):
  %cd Bachelors-project
  !git pull
  %cd ..
else:
  !git clone https://github.com/HNR1/Bachelors-project.git

/content/Bachelors-project
Already up to date.
/content


In [21]:
# calculate fid with lists of images instead of using file paths
def calc_fid(l1, l2, dims=2048):
  device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')
  block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]
  model = InceptionV3([block_idx]).to(device)
  m1, s1 = fid.calculate_activation_statistics(l1, model, min(50,len(l1)), dims, device, 1)
  m2, s2 = fid.calculate_activation_statistics(l2, model, min(50,len(l2)), dims, device, 1)
  fid_score = fid.calculate_frechet_distance(m1, s1, m2, s2)
  return fid_score

In [5]:
# parse prompts from prompts.txt
filename = "Bachelors-project/prompts.txt"
prompts = []
with open(filename, 'r', encoding='UTF-8') as file:
    while line := file.readline():
        prompts.append(line.rstrip())

In [4]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")
pipeline.enable_attention_slicing()

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [15]:
prompt = prompts[0] # photograph of an astronaut riding a horse
image_clean = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(555035)).images[0]
tomesd.apply_patch(pipeline, ratio=0.2)
image_20 = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(555035)).images[0]
tomesd.apply_patch(pipeline, ratio=0.5)
image_50 = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(555035)).images[0]
tomesd.remove_patch(pipeline)
images_clean = [image_clean]
images_20 = [image_20]
images_50 = [image_50]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [22]:
calc_fid(images_clean, images_20)

71.28360198015996

In [23]:
calc_fid(images_clean, images_50)

87.59609990240683